In [2]:
import warnings

# Import libraries
import numpy as np
import pandas as pd
from keras.layers import *
from keras.models import *
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import MultiLabelBinarizer
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.optimizers import Adam

warnings.filterwarnings('ignore')

In [3]:
pd_data = pd.read_csv('../data/multilabel_modified/multilabel_classification_clean.csv')   # reading the csv file
pd_data.head()

,Image_Name,Classes,motorcycle,truck,boat,bus,cycle,person,desert,mountains,sea,sunset,trees,sitar,ektara,flutes,tabla,harmonium
0,image1.jpg,bus person,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
1,image2.jpg,sitar,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,image3.jpg,flutes,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,image4.jpg,bus trees,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
4,image5.jpg,bus,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [4]:
train_dir = "../data/multilabel_modified/train"
val_dir = "../data/multilabel_modified/validation"
test_dir = "../data/multilabel_modified/test"

# Creating Image Data Generator for train, validation and test set
def split_and_trim(labels):
    return [label.strip() for label in labels.split(' ')]

# Apply the function to the Labels column
pd_data['Classes'] = pd_data['Classes'].apply(split_and_trim)

mlb = MultiLabelBinarizer()
multi_hot_labels = mlb.fit_transform(pd_data['Classes'])

# Add multi-hot encoded labels to DataFrame
label_columns = mlb.classes_
for i, label in enumerate(label_columns):
    pd_data[label] = multi_hot_labels[:, i]
    
y_cols = ['motorcycle', 'truck', 'boat', 'bus', 'cycle', 'person', 'desert', 'mountains', 'sea',
           'sunset', 'trees', 'sitar', 'ektara', 'flutes', 'tabla', 'harmonium']

train_gen = ImageDataGenerator(rescale = 1.0/255.0) # Normalise the data
train_image_generator = train_gen.flow_from_dataframe(
    dataframe=pd_data,
    directory=train_dir,
    x_col='Image_Name',
    y_col= y_cols,
    color_mode="rgb",
    class_mode="raw",
    target_size=(224, 224),
    batch_size=32  # to make this tutorial simple
)

val_gen = ImageDataGenerator(rescale = 1.0/255.0) # Normalise the data
val_image_generator = val_gen.flow_from_dataframe(
    dataframe=pd_data,
    directory=val_dir,
    x_col='Image_Name',
    y_col=y_cols,
    color_mode="rgb",
    class_mode="raw",
    target_size=(224, 224),
    batch_size=32  # to make this tutorial simple
)

Found 4000 validated image filenames.
Found 2000 validated image filenames.


In [9]:
# Load the InceptionV3 model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top of InceptionV3
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(len(y_cols), activation='sigmoid')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

In [17]:
# Compile the model
model.compile(optimizer=Adam(lr=0.01), loss='binary_crossentropy', metrics=['accuracy'])

In [19]:
# Train the model
history = model.fit(
    train_image_generator,
    validation_data=val_image_generator,
    epochs=3,  # Adjust the number of epochs as needed
    steps_per_epoch=len(train_image_generator),
    validation_steps=len(val_image_generator)
)

Epoch 1/3
125/125 [==============================] - 51s 400ms/step - loss: 0.0636 - accuracy: 0.8568 - val_loss: 0.6298 - val_accuracy: 0.1570
Epoch 2/3
125/125 [==============================] - 47s 377ms/step - loss: 0.0669 - accuracy: 0.8393 - val_loss: 0.4878 - val_accuracy: 0.3360
Epoch 3/3
125/125 [==============================] - 47s 377ms/step - loss: 0.0577 - accuracy: 0.8612 - val_loss: 0.4266 - val_accuracy: 0.3765


In [8]:
# Load and preprocess the input image
img_path = test_dir+'/image6005.jpg'  # Replace 'path_to_input_image' with the actual path
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array /= 255.0  # Scale pixel values to [0, 1]

# Make predictions
predictions = model.predict(img_array)

decoded_predictions = [label_columns[i] for i, pred in enumerate(predictions[0]) if pred >= 0.5]

print("Predicted labels:", decoded_predictions)


1/1 [==============================] - 3s 3s/step
Predicted labels: ['cycle']
